**TASK 1 : Managing Conversation History with Summarization**

In [22]:
!pip install openai


In [23]:
from openai import OpenAI
import os


In [39]:
# Store your API key securely in Colab
import os

# Secure input (won't show in output)
from getpass import getpass

# Prompt for key without displaying it
os.environ["GROQ_API_KEY"] = getpass("🔑 Enter your Groq API Key: ")

# Retrieve it for use
api_key = os.environ["GROQ_API_KEY"]



client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)


🔑 Enter your Groq API Key: ··········


In [32]:
class ConversationManager:
    def __init__(self, summarization_interval=3, truncation_limit=5, truncation_type="turns"):
        self.history = []
        self.run_count = 0
        self.summarization_interval = summarization_interval
        self.truncation_limit = truncation_limit
        self.truncation_type = truncation_type  # "turns" or "words"

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})

    def truncate_history(self):
        if self.truncation_type == "turns":
            return self.history[-self.truncation_limit:]
        elif self.truncation_type == "words":
            joined = " ".join([m["content"] for m in self.history])
            words = joined.split()
            truncated = " ".join(words[-self.truncation_limit:])
            return [{"role": "system", "content": truncated}]
        return self.history

    def summarize_history(self):
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "Summarize the following conversation briefly."},
                {"role": "user", "content": str(self.history)}
            ]
        )
        summary = response.choices[0].message.content
        self.history = [{"role": "system", "content": f"Summary so far: {summary}"}]

    def interact(self, user_input):
        self.add_message("user", user_input)
        truncated_history = self.truncate_history()
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=truncated_history
        )
        reply = response.choices[0].message.content

        self.add_message("assistant", reply)

        # Track runs and summarize periodically
        self.run_count += 1
        if self.run_count % self.summarization_interval == 0:
            self.summarize_history()

        return reply


In [33]:
conv = ConversationManager(summarization_interval=3, truncation_limit=4, truncation_type="turns")

print("User: Hello!")
print("Assistant:", conv.interact("Hello!"))
print("\nUser: Can you tell me about AI?")
print("Assistant:", conv.interact("Can you tell me about AI?"))
print("\nUser: What about reinforcement learning?")
print("Assistant:", conv.interact("What about reinforcement learning?"))  # triggers summarization
print("\nUser: Continue about robotics.")
print("Assistant:", conv.interact("Continue about robotics."))


User: Hello!
Assistant: Hello. It's nice to meet you. Is there something I can help you with, or would you like to chat?

User: Can you tell me about AI?
Assistant: Artificial Intelligence (AI) is a fascinating and rapidly evolving field that has been transforming numerous aspects of our lives. I'll provide you with an overview of AI, its types, applications, and some of the current trends.

**What is Artificial Intelligence (AI)?**

Artificial Intelligence refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

1. Learning: AI systems can learn from data, experiences, and interactions.
2. Problem-solving: AI systems can analyze situations, identify patterns, and make decisions.
3. Reasoning: AI systems can draw inferences, make predictions, and take actions.
4. Perception: AI systems can interpret and understand data from sensors, such as images, speech, and text.

**Types of AI:**

1. **Narrow or Weak AI**: Designed to

**TASK 2 JSON Schema Classification & Information**

In [34]:
schema = {
    "name": "extract_user_info",
    "description": "Extracts user details from a conversation.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Full name of the user"},
            "email": {"type": "string", "description": "Email address"},
            "phone": {"type": "string", "description": "Phone number"},
            "location": {"type": "string", "description": "Location or city"},
            "age": {"type": "integer", "description": "Age in years"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}


In [35]:
def extract_info_from_chat(chat_text):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are an information extraction assistant."},
            {"role": "user", "content": chat_text}
        ],
        functions=[schema],          # pass the schema
        function_call="auto"         # let the model decide when to call
    )

    # Extract structured output
    try:
        function_args = response.choices[0].message.function_call.arguments
        return function_args
    except Exception as e:
        return {"error": str(e)}


In [36]:
sample_chats = [
    "Hi, I'm Priya Sharma, my email is priya123@gmail.com, you can call me at 9876543210. I live in Mumbai and I am 22 years old.",
    "Hello, this is John Doe. You can reach me at john.doe@example.com or call at 9123456789. I'm from New Delhi, 25 years old.",
    "Hey, name's Ankit Verma, age 30. Based in Bangalore. Contact me on ankitv@gmail.com, phone 9988776655."
]

for i, chat in enumerate(sample_chats, 1):
    print(f"--- Sample Chat {i} ---")
    print(extract_info_from_chat(chat))
    print()


--- Sample Chat 1 ---
{"age":22,"email":"priya123@gmail.com","location":"Mumbai","name":"Priya Sharma","phone":"9876543210"}

--- Sample Chat 2 ---
{"age":25,"email":"john.doe@example.com","location":"New Delhi","name":"John Doe","phone":"9123456789"}

--- Sample Chat 3 ---
{"age":30,"email":"ankitv@gmail.com","location":"Bangalore","name":"Ankit Verma","phone":"9988776655"}

